# Leer registros de pruebas

## Carpeta de Pruebas

In [9]:
import pandas as pd
import os

ruta_carpeta_raiz = os.getcwd()
carpeta_leer = "Pruebas"
registros_path = os.path.join(ruta_carpeta_raiz, carpeta_leer, "registros_log.csv")
df = pd.read_csv(registros_path)

# Vista rápida
print(df.head())

# Ver cuántos experimentos guardaste
print("Total experimentos:", len(df))

# Ver qué algoritmos probaste
print(df["algoritmo"].unique())

             timestamp  carpeta                                 script  \
0  2025-09-18T19:39:24  Pruebas  20250901_PruebasEntrenamientoRF.ipynb   

      algoritmo                    dataset  clases_removidas  seed  n_train  \
0  RandomForest  PuntosMuestra_CR_2023.csv                 2    42    20439   

   n_test  n_features  num_classes  fit_seconds  pred_seconds  ms_per_sample  \
0    8760          13            6       0.9076         0.062          0.007   

       OA  F1_macro                                        system_info  
0  0.8826    0.8161  {"host": "IALB18", "user": "Fran", "os": "Wind...  
Total experimentos: 1
['RandomForest']
